In [ ]:
# install speech recognition
!pip install SpeechRecognition

# install pyaudio
!pip install pyaudio

#install pyttsx3
!pip install pyttsx3

#install torch
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

#set up transformer
!pip install transformers

# set up tensor flow
!pip install tensorflow


In [ ]:
import torch
print(torch.cuda.is_available())

True


In [1]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
tokenizer = GPT2Tokenizer.from_pretrained("af1tang/personaGPT")
model = GPT2LMHeadModel.from_pretrained("af1tang/personaGPT")
if torch.cuda.is_available():
    model = model.cuda()
## utility functions ##
flatten = lambda l: [item for sublist in l for item in sublist]

def to_data(x):
    if torch.cuda.is_available():
        x = x.cpu()
    return x.data.numpy()

def to_var(x):
    if not torch.is_tensor(x):
        x = torch.Tensor(x)
    if torch.cuda.is_available():
        x = x.cuda()
    return x

def display_dialog_history(dialog_hx):
    for j, line in enumerate(dialog_hx):
        msg = tokenizer.decode(line)
        if j %2 == 0:
            print(">> User: "+ msg)
        else:
            print("Bot: "+msg)
            print()

def generate_next(bot_input_ids, do_sample=True, top_k=10, top_p=.92,
                  max_length=1000, pad_token=tokenizer.eos_token_id):
    full_msg = model.generate(bot_input_ids, do_sample=True,
                                              top_k=top_k, top_p=top_p, 
                                              max_length=max_length, pad_token_id=tokenizer.eos_token_id)
    msg = to_data(full_msg.detach()[0])[bot_input_ids.shape[-1]:]
    return msg


In [6]:
# get personality facts for conversation
""" Personality
    """
personas = []
for i in range(6):
    response = input(">> Fact %d: "%(i+1))+ tokenizer.eos_token
    personas.append(response)
personas = tokenizer.encode(''.join(['<|p2|>'] + personas + ['<|sep|>'] + ['<|start|>']))


In [ ]:
'''
# converse for 8 turns
dialog_hx = []
for step in range(8):
    # encode the user input
    user_inp = tokenizer.encode(input(">> User: ") + tokenizer.eos_token)
    # append to the chat history
    dialog_hx.append(user_inp)
        
    # generated a response while limiting the total chat history to 1000 tokens, 
    bot_input_ids = to_var([personas + flatten(dialog_hx)]).long()
    msg = generate_next(bot_input_ids)
    dialog_hx.append(msg)
    print("Bot: {}".format(tokenizer.decode(msg, skip_special_tokens=True)))'''

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Bot: i am good. just playing with my dog.


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Bot: i don't know. do you have a dog?


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

Bot: they're good dogs


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Bot: they're nice dogs


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Bot: why not?


In [7]:
import pyttsx3

def speak_text(text):

    # Initialize the TTS engine
    engine = pyttsx3.init()

    # Set the rate of speech (optional)
    rate = engine.getProperty('rate')
    engine.setProperty('rate', rate - 50)

    # Set the voice (optional)
    # You can check available voices using engine.getProperty('voices')
    voices = engine.getProperty('voices')
    engine.setProperty('voice', voices[0].id)

    # Speak the text
    engine.say(text)

    # Wait for the speech to finish
    engine.runAndWait()


In [10]:
import speech_recognition as sr
import pyaudio
import random
# recognizer class
r = sr.Recognizer()

# listen to audio

# process audio okish
while True:
    with sr.Microphone() as source:
        print("listening")
        audio = r.listen(source,5, phrase_time_limit=5)
    try:
        r.recognize_google(audio)
        text = r.recognize_google(audio)
        print("you speak: ", text) # send text to processing
        # converse for 8 turns
        dialog_hx = []
        # encode the user input
        user_inp = tokenizer.encode(text + tokenizer.eos_token)
        # append to the chat history
        dialog_hx.append(user_inp)
            
        # generated a response while limiting the total chat history to 1000 tokens, 
        bot_input_ids = to_var([personas + flatten(dialog_hx)]).long()
        msg = generate_next(bot_input_ids)
        dialog_hx.append(msg)
        text_from_model = "{}".format(tokenizer.decode(msg, skip_special_tokens=True))
        random_number = random.randint(1,4)
        if random_number == 4:
            dict_of_random_responses = {1 : "pog", 2 : "pogchamp", 3 : "cheater", 4 : "Kappa", 5 : "LOL"}
            response_number = random.randint(1,5)
            text_from_model = dict_of_random_responses[response_number]
        speak_text(text_from_model)
        print("Bot : " + text_from_model)
        #print("Bot: {}".format(tokenizer.decode(msg, skip_special_tokens=True)))
        #speak_text(text)
    except sr.UnknownValueError:
        response = "What are you talking about"
        speak_text(response)
    except sr.RequestError as e:
        response = "What are you talking about"
        speak_text(response)
        
    
    

listening


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


you speak:  can you hear me right now
Bot: no i cant, my voice is terrible
listening


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


you speak:  why is your voice terrible
Bot: i'm not good with computers
listening


...

you speak:  sometimes
Bot: what is your name?
listening


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


you speak:  I don't have a name
Bot: i am a greg my middle name is
listening


Exception ignored in: <function BSTR.__del__ at 0x000001EEFA7AEF80>
Traceback (most recent call last):
  File "c:\Users\ASUS\anaconda3\lib\site-packages\comtypes\__init__.py", line 699, in __del__
    def __del__(self, _free=windll.oleaut32.SysFreeString):
KeyboardInterrupt: 


listening
